(https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=원달러환율&ackey=5r1iedfw)

- protocol: https
- domain: search.naver.com
- path: search.naver
- parameters: where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=원달러환율&ackey=5wft1q6p

```python
{
    'where': 'nexearch',
    'sm': 'top_hty',
    'fbm': 0,
    'ie': 'utf8',
    'query': '원달러환율',
    'ackey': '5wft1q6p',
}

In [55]:
import requests
from bs4 import BeautifulSoup
import lxml

In [ ]:
search_list = ['후쿠오카', '치앙마이', '다낭']
result = []
for i in search_list:
    response = requests.get('https://search.naver.com/search.naver?sm=whl_nht&query=' + i)
    response.status_code
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    box = soup.find('ul', attrs={'class':'lst_related_srch _list_box'})

    term = []

    divs = box.select('li > a > div')
    for div in divs:
        term.append(div.text)

    result.append({'search_term':i, 'related_term':term})

result


In [ ]:
#함수로 바꿔보기

def rel_search(list):
    result = []
    
    for i in list:
        response = requests.get('https://search.naver.com/search.naver?sm=whl_nht&query=' + i)
        response.status_code
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        box = soup.find('ul', attrs={'class':'lst_related_srch _list_box'})

        term = []

        divs = box.select('li > a > div')
        for div in divs:
            term.append(div.text)

        result.append({'search_term':i, 'related_term':term})
        
    return result

list = ['삿포로', '치앙마이', '오키나와']
rel_search(list)

In [ ]:
#연관검색어 한 번 더 찾기
def rel_search(list):
    result = []
    
    for i in list:
        response = requests.get('https://search.naver.com/search.naver?sm=whl_nht&query=' + i)
        response.status_code
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        box = soup.find('ul', attrs={'class':'lst_related_srch _list_box'})

        term = []

        divs = box.select('li > a > div')
        for div in divs:
            term.append(div.text)

        result.append({'search_term':i, 'related_term':term})
        
    return result


list = ['치앙마이']
rel_term = rel_search(list)[0]['related_term']
rel_search(rel_term)

    

In [38]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

selenium.__version__

'4.36.0'

In [ ]:
options = Options()
options.add_argument("disable-blink-features=AutomationControlled") # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"]) #자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False) # 자동화 확장기능 사용 안 함
# https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome
options.add_argument("--user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36")
# options.add_argument("--headless") # headless mode
ch_driver = webdriver.Chrome(options=options)

In [ ]:
jg_url = 'https://cafe.naver.com/joonggonara'
ch_driver.get(jg_url) # 페이지 이동

In [ ]:
query_input = ch_driver.find_element(By.ID, "topLayerQueryInput") # 요소 찾기
search_term = '맥북'
query_input.send_keys(search_term) # 키보드 입력

In [ ]:
# 1. 검색어 입력후 엔터
# search_input.send_keys(Keys.ENTER)

# 2. 검색버튼 누르기
# search_btn = ch_driver.find_element(By.XPATH, "/html/body/div[3]/div/div[6]/div[1]/div[2]/form/button")
# search_btn.click()

#3. 자바스크립트 실행
# ch_driver.execute_script('searchBoard();')

In [ ]:
posts = ch_driver.find_elements(By.CSS_SELECTOR, 'table.atricle-table tr')

In [ ]:
#페이지 직접 이동
page_to = 2 #페이지 수 지정
btns = ch_driver.find_elements(By.CSS_SELECTOR, 'div.Paginataion > button')

for btn in btns:
    try:
        if int(btn.text) == page_to :
            btn.click()
    except:
        break

In [ ]:
#url로 이동
post_list = []
for i in range(1,10+1):
    search_result_uri = f'https://cafe.naver.com/f-e/cafes/10050146/menus/0?viewType=L&ta=ARTICLE_COMMENT&page={i}&q=맥북'
    ch_driver.get(search_result_uri) # 각 페이지를 방문했을 때
    sleep(2) # 페이지 로딩 중 정보수집하는 것을 막기위한 기다림
    posts = ch_driver.find_elements(By.CSS_SELECTOR, 'table.article-table tr') # 전체 페이지에서 게시물 단위를 일단 분할해두고
    for post in posts[1:]: # 각 게시물 단위에 대해
        post_no = post.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text
        post_title = post.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').text
        post_link = post.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').get_attribute('href')
        # 작성시간
        post_time = post.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text
        # 조회수 구해보기
        post_views = post.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text
        post_list.append({
            'no': post_no,
            'title': post_title,
            'link': post_link,
            'datetime': post_time,
            'views': post_views,
        })
    print(f'{i}페이지 작업 완료')
    
    sleep(1) #파이썬 인터프리터를 1초간 멈춤


wait

In [ ]:
ch_driver.implicitly_wait(2) #웹드라이버가 2초간 멈춤

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
wait = WebDriverWait(ch_driver, 4)
# 기본적으로 대기할 시간 설정
wait.until(EC.visibility_of_element_located(By.CSS_SELECTOR, 'table.article-table'))
# 위 조건을 만족할 경우 (table이 페이지에 보여질 경우) 멈춤을 끝낸다

window

In [ ]:
ch_driver.current_window_handle # 현재 작업창 확인

In [ ]:
ch_driver.execute_script('window.opne();') # 새 탭 띄우기

In [ ]:
ch_driver.current_window_handles # 지금 존재하는 탭 윈도우 리스트 확인

In [ ]:
ch_driver.switch_to.window(window_name=ch_driver.window_handles[1]) #탭 전환

In [ ]:
ch_driver.get() # 두번째 탭에서 페이지 띄우기

In [ ]:
ch_driver.close() # 작업중이던 탭 종료, 남은 탭으로 이동 필요

In [ ]:
ch_driver.switch_to.window(window_name=ch_driver.window_handles[0])

screen shot

In [ ]:
ch_driver.fullscreen_window() # 창 최대화
ch_driver.save_screenshot('./screenshot.png') #현재 위치에 파일 저장 (경로, 확장자 꼭 적기)

In [ ]:
ch_driver.close()